In [24]:
import pandas as pd
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
import pickle
from CorrectFeatures import correct_SCT_data, correct_SCT_labels

In [25]:
training_data = correct_SCT_data(pd.read_csv("Data/Scotland/scottish_data_training.csv"))
training_labels = correct_SCT_labels(pd.read_csv("Data/Scotland/scottish_labels_training.csv"))

In [26]:
Y = training_labels
# Y = training_labels.drop(columns = ['attendance', 'number_over_4_hours',
#        'number_over_8_hours', 'percentage_within_8_hours',
#        'number_over_12_hours', 'percentage_within_12_hours',
#        'number_under_4_hours', 'number_4_hours_8_hours',
#        'number_8_hours_12_hours', 'percentage_within_4_hours'])

In [27]:
X = training_data
# X["region_east"] = 1.0 *(X["region"] == "East")
# X["region_west"] =  1.0 *(X["region"] == "West")
# X["region_north"] =  1.0 *(X["region"] == "North")
# X = X.drop(columns=["region", "moon_phase_name"])
# for col in X.columns:
#     X[col] = X[col].astype(str).str.replace(',', '').astype(float)
#     if X[col].isna().any():
#         X[col] = X[col].fillna(0)

In [28]:
model_4_hours = MLPRegressor(hidden_layer_sizes=(100, 100, 100, 100, 100, 100, 100, 100,), activation="tanh")
model_8_hours = MLPRegressor(hidden_layer_sizes=(100, 100, 100, 100, 100, 100, 100, 100,),activation="tanh")
model_12_hours = MLPRegressor(hidden_layer_sizes=(100, 100, 100, 100, 100, 100, 100, 100,),activation="tanh")
model_long_hours = MLPRegressor(hidden_layer_sizes=(100, 100, 100, 100, 100, 100, 100, 100,),activation="tanh")


In [29]:
model_4_hours.fit(X,Y.proportion_under_4_hours)
model_8_hours.fit(X,Y.proportion_4_hours_8_hours)
model_12_hours.fit(X,Y.proportion_8_hours_12_hours)
model_long_hours.fit(X,Y.proportion_over_12_hours)


MLPRegressor(activation='tanh',
             hidden_layer_sizes=(100, 100, 100, 100, 100, 100, 100, 100))

In [30]:
(mean_absolute_error(Y.proportion_under_4_hours, model_4_hours.predict(X)),
 mean_absolute_error(Y.proportion_4_hours_8_hours, model_8_hours.predict(X)),
 mean_absolute_error(Y.proportion_8_hours_12_hours, model_12_hours.predict(X)),
 mean_absolute_error(Y.proportion_over_12_hours, model_long_hours.predict(X)))

(0.054421056397506815,
 0.06353148360897856,
 0.009402842414109052,
 0.009413181819064979)

In [31]:
testing_data = correct_SCT_data(pd.read_csv("Data/Scotland/scottish_data_testing.csv"))
testing_labels = correct_SCT_labels(pd.read_csv("Data/Scotland/scottish_labels_testing.csv"))

In [32]:
Y_test = testing_labels
X_test = testing_data

In [34]:
(mean_absolute_error(Y_test.proportion_under_4_hours, model_4_hours.predict(X_test)),
 mean_absolute_error(Y_test.proportion_4_hours_8_hours, model_8_hours.predict(X_test)),
 mean_absolute_error(Y_test.proportion_8_hours_12_hours, model_12_hours.predict(X_test)),
 mean_absolute_error(Y_test.proportion_over_12_hours, model_long_hours.predict(X_test)))

(0.053415537341286186,
 0.06331566569031688,
 0.009101714887160777,
 0.009317283252826948)

In [35]:
# def divide_by_sum(tuple_):
#     sum_ = sum(tuple_)
#     return [e/sum_ for e in tuple_]
# normed_predictions = [list(l) for l in
#                       zip(*map(divide_by_sum,
#                                zip(model_4_hours.predict(X_test),
#                                    model_8_hours.predict(X_test),
#                                    model_12_hours.predict(X_test),
#                                    model_long_hours.predict(X_test))))
#                       ]
# (mean_absolute_error(Y_test.proportion_under_4_hours, normed_predictions[0]),
#  mean_absolute_error(Y_test.proportion_4_hours_8_hours, normed_predictions[1]),
#  mean_absolute_error(Y_test.proportion_8_hours_12_hours, normed_predictions[2]),
#  mean_absolute_error(Y_test.proportion_over_12_hours, normed_predictions[3]))

In [36]:
model_bytes = pickle.dumps((model_4_hours,
                            model_8_hours,
                            model_12_hours,
                            model_long_hours))
print(len(model_bytes))
with open("Models/TrainingMLPScotlandModel.pkl", 'wb') as save_file:
    save_file.write(model_bytes)


11750722


In [37]:
model_4_hours_loaded,\
model_8_hours_loaded,\
model_12_hours_loaded,\
model_long_hours_loaded = None, None, None, None
with open("Models/TrainingMLPScotlandModel.pkl", 'rb') as save_file:
    model_bytes = save_file.read()
    print(len(model_bytes))
    model_4_hours_loaded,\
    model_8_hours_loaded,\
    model_12_hours_loaded,\
    model_long_hours_loaded = pickle.loads(model_bytes)

11750722
